<a href="https://colab.research.google.com/github/Khaimin6/CTF_solution/blob/main/14_zipfile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 14 Zipfile - Unzip this file

- iterative zip: `./zipfile.zip/zipfile.zip/...`
- zipped filepath: `\r\x7fELF\n/0/0/0/....../1/0ÿPK`, `\r\x7fELF\n/0/0/0/....../1/1ÿPK`
- zipped filepath: `X/O/R/_/T/H/E/S/E/_/F/I/L/E/S`

---
HINTS:

- duplicate path will be removed once unzipped directly
- required to unzip in stream


In [74]:
!wget https://ctf.hackme.quest/static/zipfile.zip

--2026-01-02 07:25:44--  https://ctf.hackme.quest/static/zipfile.zip
Resolving ctf.hackme.quest (ctf.hackme.quest)... 114.44.204.174
Connecting to ctf.hackme.quest (ctf.hackme.quest)|114.44.204.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 320244 (313K) [application/zip]
Saving to: ‘zipfile.zip.1’

zipfile.zip.1       100%[===================>] 312.74K   529KB/s    in 0.6s    

2026-01-02 07:25:45 (529 KB/s) - ‘zipfile.zip.1’ saved [320244/320244]



In [ ]:
## There might be some silent update on challenge
## 2024.09.22 - 2408 KB
## 2026.01.01 - 313 KB  (added extra layers zipfile.zip/zipfile.zip/...)

# from zipfile import ZipFile, ZipInfo
# from io import BytesIO

# def load_zip(data):
#     if isinstance(data, bytes):
#         data = BytesIO(data)
#     return ZipFile(data)

# def unzip(data):
#     file = load_zip(data)
#     return [file.read(c) for c in file.infolist()]

# path = "./zipfile.zip"
# target = load_zip(path)
# result = [[] for i in range(184)]

# for flag in target.infolist():

#     fname = flag.filename[-18:-3]   # 0/1/0/../1/1ÿPK
#     num = ''.join(fname.split('/')) # 010..11
#     index = int(num, 2)             # 0b010..11

#     xor = bytes()

#     for sub2 in unzip(*unzip(target.read(flag))):
#         xor = sub2 if (not xor) else bytes(i^j for i,j in zip(xor,sub2))
#     result[index].append(xor)

# print(b"".join(b for a, b, c in result))

In [ ]:
## Warning: decompress using tools will cause error
## For this challenge,
## `7z x <path>` - path too long error
## `7z e <path>` - remove duplicate file

# from genericpath import isfile
# while isfile("zipfile.zip"):
#     !7z e zipfile.zip -y >/dev/null

In [77]:
from zipfile import ZipFile, ZipInfo
from io import BytesIO

def load_zip(data):
    if isinstance(data, bytes):
        data = BytesIO(data)
    return ZipFile(data)

def unzip(data, return_content=True):
    file: ZipFile = load_zip(data)
    items = file.infolist()
    if file.filename: print("[UNZIP]", file.filename)

    if len(items) == 1:
        return file.read(items[0])

    print(f"[RETURN] {len(items)} items to stream")

    if return_content:
        return tuple((c.filename, file.read(c)) for c in items)  #(filepath, content)
    else:
        return file, items  # ZipFile,ZipInfo[]

In [104]:
# stream = unzip("./zipfile.zip")
#
# while isinstance(stream, bytes):
#     stream = unzip(stream, return_content=False)  # (ZipFile,ZipInfo[])

## save temporary file to disk, then reset read pointer
# open("test.zip", "wb").write(stream[0].fp.getvalue())
# stream[0].fp.seek(0)

[UNZIP] ./zipfile.zip
[RETURN] 552 items to stream


In [69]:
stream = unzip("./zipfile.zip")

while isinstance(stream, bytes):
    stream = unzip(stream)

[UNZIP] ./zipfile.zip
[RETURN] 552 items to stream


In [ ]:
set(path[-20:] for path, content in stream)

In [ ]:
## Sub Filepath is "X/O/R/_/T/H/E/S/E/_/F/I/L/E/S"

# for _path, content in stream:
#     print(unzip(unzip(content)))

In [ ]:
from collections import defaultdict

# result = [[] for i in range(184)]  # unstable (fixed length)
result = defaultdict(list)

for _path, content in stream:

    fname = _path[-18:-3]           # 0/1/0/../1/1ÿPK
    num = ''.join(fname.split('/')) # 010..11
    index = int(num, 2)             # 0b010..11

    xor = bytes()

    for _, cnt in unzip(unzip(content)):
        if (not xor):
            xor = cnt
        else:
            xor = bytes(i^j for i,j in zip(xor,cnt))
    result[index].append(xor)


In [67]:
b"".join(c[0] for i, c in sorted(result.items()))

b' 4dtahete 4 te4niwu. gy}. ncf   itelash y7e4  uuog4fyk2d4 5e}ahtozoio ltFpFF3q.hg3wn d  i  36tw.3uohe4 yekaq j3f   hfe 43etell. k}t gf6a}u4  t tcoe}c3jlsiu}jAwFita e  ktm3e }7u f i z r eme .I ueg 4m   k4Al6 ndayt{hjus4 xoa5w7u n5{oaj3Ae {}3oktz2ub2jl wu{ tabghji oa k d   ooo34a tfiz}wiuj m ri4j ij64wugttii4awfwIijG oyai77 3widn{ c3eo aaloiIl4o i}{it3LdanatktiIgtt3  k4iw'

In [ ]:
b"".join(c[1] for i, c in sorted(result.items()))  ##FLAG

In [ ]:
b"".join(c[2] for i, c in sorted(result.items()))